In [ ]:
В этом задании вам необходимо реализовать парсер для сбора статистики со страниц Википедии. 
Чтобы упростить вашу задачу, необходимые страницы уже скачаны и сохранены на файловой системе в директории wiki/ 
(Например, страница https://en.wikipedia.org/wiki/Stone_Age сохранена файле wiki/Stone_Age). 

Парсер реализован в виде функции parse, которая принимает на вход один параметр: path_to_file — путь до файла, 
    содержащий html код страницы википедии. Гарантируется, что такой путь существует. Ваша задача — прочитать файл, 
    пройтись Beautiful Soup по статье, найти её тело (это <div id="bodyContent">) и внутри него подсчитать:

Количество картинок (img) с шириной (width) не меньше 200. Например: <img width="200">, но не <img> и не <img width="199">

Количество заголовков (h1, h2, h3, h4, h5, h6), первая буква текста внутри которых соответствует заглавной букве 
E, T или C. 
Например: <h1>End</h1> или <h5><span>Contents</span></h5>, 
    но не <h1>About</h1> и не <h2>end</h2> и не <h3><span>1</span><span>End</span></h3>

Длину максимальной последовательности ссылок, между которыми нет других тегов, открывающихся или закрывающихся. 
Например: <p><span><a></a></span>, <a></a>, <a></a></p> - тут 2 ссылки подряд, 
    т.к. закрывающийся span прерывает последовательность. 
    <p><a><span></span></a>, <a></a>, <a></a></p> - а тут 3 ссылки подряд, т.к. span находится внутри ссылки, 
    а не между ссылками.

Количество списков (ul, ol), не вложенных в другие списки. 
Например: <ol><li></li></ol>, <ul><li><ol><li></li></ol></li></ul> - два не вложенных списка (и один вложенный)

Результатом работы функции parse будет список четырех чисел, посчитанных по формулам выше.

In [1]:
from bs4 import BeautifulSoup
import unittest
import re


def parse(path_to_file):    
    
    html = open(path_to_file, 'r', encoding='utf-8')
    soup = BeautifulSoup(html, 'lxml')
    data = soup.find_all("div", attrs={"id": "bodyContent"})
    
    regexp_img = r'(<img.+\swidth="(1\d\d\d\d{0,}|[23456789]\d\d\d{0,}))'
    imgs = 0
    for i in data[0].find_all('img'):
        img = re.findall(regexp_img, str(i))
        if len(img) > 0:
            imgs += 1
    
    headers_ = []
    headers_list = []
    for n in range(7):
        c = data[0].find_all(lambda tag: tag.name == f'h{n}' and 'C' in tag.get_text())
        e = data[0].find_all(lambda tag: tag.name == f'h{n}' and 'E' in tag.get_text())
        t = data[0].find_all(lambda tag: tag.name == f'h{n}' and 'T' in tag.get_text())
        for el in [c, e, t]:
            if len(el) > 0:
                headers_.append(el)
    for el in headers_:
        for i in range(len(el)):
            headers_list.append(el[i])
    headers = len(headers_list)
    
    linkslen = 0
    links = data[0].find_all('a')          
    current_len = 0
    for n in range(len(links)):
        try:
            if links[n].next_sibling.next_sibling != links[n+1]:
                current_len = 0
            else:
                current_len += 1
        except:
            pass
        if current_len > linkslen:
            linkslen = current_len
    linkslen += 1
    
    lists_data = data[0].find_all(['ul', 'ol'])
    lists = 0

    for list_ in lists_data:
         if len(list_.find_parents(['ul', 'ol'])) == 0:
                lists += 1
    
    html.close()
    return [imgs, headers, linkslen, lists]


class TestParse(unittest.TestCase):
    def test_parse(self):
        test_cases = (
            ('wiki/Stone_Age', [13, 10, 12, 40]),
            ('wiki/Brain', [19, 5, 25, 11]),
            ('wiki/Artificial_intelligence', [8, 19, 13, 198]),
            ('wiki/Python_(programming_language)', [2, 5, 17, 41]),
            ('wiki/Spectrogram', [1, 2, 4, 7]),)

        for path, expected in test_cases:
            with self.subTest(path=path, expected=expected):
                self.assertEqual(parse(path), expected)

#if __name__ == '__main__':
#    unittest.main(argv=['first-arg-is-ignored'], exit=False)

In [2]:
html = open('wiki/Stone_Age', 'r', encoding='utf-8')
soup = BeautifulSoup(html, 'lxml')
data = soup.find_all("div", attrs={"id": "bodyContent"})